<a href="https://www.kaggle.com/code/rogernickanaedevha/neural-ode-model?scriptVersionId=272380189" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


# Paper 4: Neural ODE-Point Process Integration for Real-Time Adaptive Network Defense
## Target: IEEE Transactions on Neural Networks and Learning Systems
## Author: Roger Nick Anaedevha


In [ ]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchdiffeq import odeint, odeint_adjoint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, List, Dict, Optional
import warnings
import os
import kagglehub
from tqdm import tqdm
import time
from collections import defaultdict
from scipy.stats import norm
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO, Predictive
from pyro.optim import Adam as PyroAdam

warnings.filterwarnings('ignore')

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ========================= Neural ODE Components =========================

class ODEFunc(nn.Module):
    """Neural ODE dynamics function"""
    
    def __init__(self, hidden_dim, n_layers=3):
        super().__init__()
        
        layers = []
        for i in range(n_layers):
            if i == 0:
                layers.append(nn.Linear(hidden_dim + 1, hidden_dim))  # +1 for time
            else:
                layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.Tanh())
            
        self.net = nn.Sequential(*layers)
        self.hidden_dim = hidden_dim
        
    def forward(self, t, h):
        """
        Args:
            t: Current time
            h: Hidden state [batch_size, hidden_dim]
        """
        # Concatenate time to hidden state
        t_vec = torch.ones(h.shape[0], 1).to(h.device) * t
        h_t = torch.cat([h, t_vec], dim=1)
        
        # Compute dynamics
        dh_dt = self.net(h_t)
        
        return dh_dt

class BayesianNeuralODE(nn.Module):
    """Bayesian Neural ODE for network dynamics modeling"""
    
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers=3):
        super().__init__()
        
        # Feature encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # ODE dynamics
        self.ode_func = ODEFunc(hidden_dim, n_layers)
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
        # Uncertainty parameters
        self.log_noise = nn.Parameter(torch.zeros(1))
        
    def forward(self, x, t_span):
        """
        Args:
            x: Input features [batch_size, input_dim]
            t_span: Time points for ODE integration
        """
        # Encode input
        h0 = self.encoder(x)
        
        # Solve ODE
        h_t = odeint_adjoint(
            self.ode_func,
            h0,
            t_span,
            method='dopri5',
            rtol=1e-3,
            atol=1e-4
        )
        
        # Select final time point
        h_final = h_t[-1]
        
        # Decode
        output = self.decoder(h_final)
        
        return output, h_final
    
    def sample_trajectory(self, x, t_span, n_samples=10):
        """Sample multiple trajectories for uncertainty estimation"""
        trajectories = []
        
        for _ in range(n_samples):
            # Add noise to initial condition
            h0 = self.encoder(x)
            noise = torch.randn_like(h0) * torch.exp(self.log_noise)
            h0_noisy = h0 + noise
            
            # Solve ODE
            h_t = odeint(
                self.ode_func,
                h0_noisy,
                t_span,
                method='dopri5'
            )
            
            trajectories.append(h_t)
            
        return torch.stack(trajectories)

# ========================= Point Process Components =========================

class HawkesProcess(nn.Module):
    """Multivariate Hawkes Process for attack event modeling"""
    
    def __init__(self, n_types, hidden_dim=64):
        super().__init__()
        
        self.n_types = n_types
        
        # Base intensity
        self.mu = nn.Parameter(torch.ones(n_types) * 0.1)
        
        # Excitation matrix
        self.alpha = nn.Parameter(torch.ones(n_types, n_types) * 0.1)
        
        # Decay parameters
        self.beta = nn.Parameter(torch.ones(n_types) * 1.0)
        
        # Neural intensity modulation
        self.intensity_net = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, n_types),
            nn.Softplus()
        )
        
    def compute_intensity(self, t, history, hidden_state=None):
        """
        Compute conditional intensity at time t
        
        Args:
            t: Current time
            history: List of (time, type) tuples
            hidden_state: Optional hidden state from Neural ODE
        """
        intensity = self.mu.clone()
        
        # Add self-excitation from history
        for t_i, m_i in history:
            if t_i < t:
                dt = t - t_i
                intensity += self.alpha[m_i, :] * torch.exp(-self.beta * dt)
                
        # Modulate with hidden state if available
        if hidden_state is not None:
            modulation = self.intensity_net(hidden_state)
            intensity = intensity * modulation.squeeze()
            
        return torch.clamp(intensity, min=1e-6)
    
    def log_likelihood(self, events, T):
        """
        Compute log-likelihood of event sequence
        
        Args:
            events: List of (time, type) tuples
            T: Observation window
        """
        ll = 0
        
        for i, (t_i, m_i) in enumerate(events):
            # Intensity at event time
            history = events[:i]
            lambda_i = self.compute_intensity(t_i, history)
            ll += torch.log(lambda_i[m_i])
            
        # Integral term (compensator)
        compensator = self.compute_compensator(events, T)
        ll -= compensator
        
        return ll
    
    def compute_compensator(self, events, T):
        """Compute integrated intensity (compensator)"""
        compensator = self.mu.sum() * T
        
        for t_i, m_i in events:
            # Contribution from each event
            integral = self.alpha[m_i, :].sum() / self.beta * (1 - torch.exp(-self.beta * (T - t_i)))
            compensator += integral.sum()
            
        return compensator
    
    def sample_next_event(self, t_current, history, max_time=10.0):
        """Sample next event using thinning algorithm"""
        t = t_current
        
        while t < max_time:
            # Upper bound on intensity
            lambda_max = self.compute_intensity(t, history).sum() * 1.5
            
            # Sample waiting time
            dt = torch.distributions.Exponential(lambda_max).sample()
            t = t + dt
            
            if t > max_time:
                break
                
            # Accept/reject
            lambda_t = self.compute_intensity(t, history)
            u = torch.rand(1)
            
            if u < lambda_t.sum() / lambda_max:
                # Accept event
                # Sample type
                probs = lambda_t / lambda_t.sum()
                m = torch.multinomial(probs, 1).item()
                return t, m
                
        return None, None

# ========================= Neural ODE-Point Process Integration =========================

class NeuralODEPointProcess(nn.Module):
    """Integrated Neural ODE-Point Process model"""
    
    def __init__(self, input_dim, hidden_dim, n_attack_types):
        super().__init__()
        
        # Neural ODE for continuous dynamics
        self.neural_ode = BayesianNeuralODE(
            input_dim, hidden_dim, n_attack_types
        )
        
        # Hawkes process for discrete events
        self.point_process = HawkesProcess(n_attack_types, hidden_dim)
        
        # Coupling network
        self.coupling = nn.Sequential(
            nn.Linear(hidden_dim + n_attack_types, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        self.hidden_dim = hidden_dim
        self.n_attack_types = n_attack_types
        
    def forward(self, x, t_span, events=None):
        """
        Forward pass combining ODE and point process
        
        Args:
            x: Input features
            t_span: Time points for integration
            events: Optional event history
        """
        # Get ODE trajectory
        output, h_final = self.neural_ode(x, t_span)
        
        # Compute point process intensity
        if events is not None:
            intensities = []
            for t in t_span:
                intensity = self.point_process.compute_intensity(
                    t, events, h_final
                )
                intensities.append(intensity)
            intensities = torch.stack(intensities)
        else:
            intensities = None
            
        return output, h_final, intensities
    
    def compute_elbo(self, x, y, t_span, events, n_samples=10):
        """Compute evidence lower bound for variational inference"""
        
        # Sample multiple trajectories
        log_probs = []
        kl_divs = []
        
        for _ in range(n_samples):
            # Forward pass
            output, h_final, intensities = self.forward(x, t_span, events)
            
            # Classification likelihood
            log_p_y = -F.cross_entropy(output, y, reduction='none')
            log_probs.append(log_p_y)
            
            # Point process likelihood
            if events is not None:
                pp_ll = self.point_process.log_likelihood(events, t_span[-1])
                log_probs[-1] = log_probs[-1] + 0.1 * pp_ll
                
            # KL divergence (simplified - assuming Gaussian prior)
            kl = 0.5 * torch.mean(h_final ** 2)
            kl_divs.append(kl)
            
        # Average over samples
        log_prob = torch.stack(log_probs).mean(0)
        kl_div = torch.stack(kl_divs).mean()
        
        elbo = log_prob.mean() - kl_div
        
        return -elbo  # Return negative ELBO as loss

# ========================= Variational Inference =========================

class VariationalInference:
    """Variational inference for Bayesian Neural ODE-PP"""
    
    def __init__(self, model, device):
        self.model = model.to(device)
        self.device = device
        
    def model_fn(self, x, y=None):
        """Pyro model for VI"""
        # Priors on ODE parameters
        ode_weight_prior = dist.Normal(0, 1)
        ode_bias_prior = dist.Normal(0, 1)
        
        # Priors on point process parameters
        mu_prior = dist.Gamma(1, 1)
        alpha_prior = dist.Gamma(1, 1)
        beta_prior = dist.Gamma(1, 1)
        
        # Sample parameters
        pyro.module("neural_ode_pp", self.model)
        
        # Likelihood
        if y is not None:
            with pyro.plate("data", len(x)):
                output, _, _ = self.model(x, torch.linspace(0, 1, 10))
                pyro.sample("obs", dist.Categorical(logits=output), obs=y)
                
    def guide_fn(self, x, y=None):
        """Pyro guide (variational distribution) for VI"""
        # Variational parameters
        pyro.module("neural_ode_pp", self.model)
        
    def train_svi(self, train_loader, epochs=10):
        """Train using stochastic variational inference"""
        optimizer = PyroAdam({"lr": 1e-3})
        svi = SVI(self.model_fn, self.guide_fn, optimizer, loss=Trace_ELBO())
        
        losses = []
        for epoch in range(epochs):
            epoch_loss = 0
            for x, y in train_loader:
                x, y = x.to(self.device), y.to(self.device)
                loss = svi.step(x, y)
                epoch_loss += loss
                
            avg_loss = epoch_loss / len(train_loader)
            losses.append(avg_loss)
            
            if (epoch + 1) % 5 == 0:
                print(f"Epoch {epoch+1}/{epochs}, ELBO Loss: {avg_loss:.4f}")
                
        return losses

# ========================= Real-Time Adaptive Learning =========================

class RealTimeAdapter:
    """Real-time adaptive learning for streaming data"""
    
    def __init__(self, model, device, buffer_size=1000):
        self.model = model.to(device)
        self.device = device
        self.buffer_size = buffer_size
        
        # Experience replay buffer
        self.buffer_x = []
        self.buffer_y = []
        self.buffer_events = []
        
        # Online statistics
        self.n_seen = 0
        self.accuracy_window = []
        
    def update(self, x, y, events=None):
        """Online update with new sample"""
        # Add to buffer
        self.buffer_x.append(x)
        self.buffer_y.append(y)
        if events is not None:
            self.buffer_events.append(events)
            
        # Maintain buffer size
        if len(self.buffer_x) > self.buffer_size:
            self.buffer_x.pop(0)
            self.buffer_y.pop(0)
            if len(self.buffer_events) > 0:
                self.buffer_events.pop(0)
                
        self.n_seen += 1
        
        # Periodic adaptation
        if self.n_seen % 100 == 0:
            self.adapt()
            
    def adapt(self):
        """Adapt model with buffered data"""
        if len(self.buffer_x) < 10:
            return
            
        # Convert buffer to tensors
        X = torch.stack(self.buffer_x)
        y = torch.stack(self.buffer_y)
        
        # Quick fine-tuning
        optimizer = optim.Adam(self.model.parameters(), lr=1e-4)
        
        for _ in range(5):  # Few gradient steps
            optimizer.zero_grad()
            
            t_span = torch.linspace(0, 1, 10).to(self.device)
            output, _, _ = self.model(X, t_span)
            
            loss = F.cross_entropy(output, y)
            loss.backward()
            optimizer.step()
            
    def predict_with_uncertainty(self, x, n_samples=10):
        """Predict with uncertainty quantification"""
        self.model.eval()
        
        predictions = []
        with torch.no_grad():
            for _ in range(n_samples):
                t_span = torch.linspace(0, 1, 10).to(self.device)
                output, _, _ = self.model(x.unsqueeze(0), t_span)
                prob = F.softmax(output, dim=1)
                predictions.append(prob)
                
        predictions = torch.stack(predictions)
        
        # Mean and uncertainty
        mean_pred = predictions.mean(0)
        uncertainty = predictions.std(0)
        
        return mean_pred, uncertainty

# ========================= Evaluation Framework =========================

class RealTimeEvaluator:
    """Comprehensive evaluation for real-time adaptive system"""
    
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.results = {}
        
    def evaluate_detection_performance(self, test_loader):
        """Evaluate intrusion detection performance"""
        print("\n=== Detection Performance ===")
        
        self.model.eval()
        all_preds = []
        all_labels = []
        all_probs = []
        
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(self.device), y.to(self.device)
                
                t_span = torch.linspace(0, 1, 10).to(self.device)
                output, _, _ = self.model(x, t_span)
                
                probs = F.softmax(output, dim=1)
                preds = torch.argmax(output, dim=1)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(y.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())
                
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')
        
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1 Score: {f1:.4f}")
        
        self.results['accuracy'] = accuracy
        self.results['f1'] = f1
        
        return self.results
    
    def evaluate_uncertainty_calibration(self, test_loader, n_samples=20):
        """Evaluate uncertainty calibration"""
        print("\n=== Uncertainty Calibration ===")
        
        self.model.eval()
        
        confidences = []
        accuracies = []
        
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(self.device), y.to(self.device)
                
                # Get predictions with uncertainty
                preds_list = []
                for _ in range(n_samples):
                    t_span = torch.linspace(0, 1, 10).to(self.device)
                    output, _, _ = self.model(x, t_span)
                    probs = F.softmax(output, dim=1)
                    preds_list.append(probs)
                    
                # Average predictions
                mean_probs = torch.stack(preds_list).mean(0)
                pred_class = torch.argmax(mean_probs, dim=1)
                confidence = mean_probs.max(dim=1)[0]
                
                # Check accuracy
                correct = (pred_class == y).float()
                
                confidences.extend(confidence.cpu().numpy())
                accuracies.extend(correct.cpu().numpy())
                
        # Calibration error
        confidences = np.array(confidences)
        accuracies = np.array(accuracies)
        
        # ECE calculation
        n_bins = 10
        bin_boundaries = np.linspace(0, 1, n_bins + 1)
        ece = 0
        
        for i in range(n_bins):
            mask = (confidences > bin_boundaries[i]) & (confidences <= bin_boundaries[i+1])
            if mask.sum() > 0:
                bin_acc = accuracies[mask].mean()
                bin_conf = confidences[mask].mean()
                ece += mask.sum() * np.abs(bin_acc - bin_conf)
                
        ece /= len(confidences)
        
        print(f"Expected Calibration Error: {ece:.4f}")
        
        self.results['ece'] = ece
        
        return self.results
    
    def evaluate_temporal_performance(self, test_loader):
        """Evaluate temporal modeling performance"""
        print("\n=== Temporal Performance ===")
        
        # Simulate temporal attack patterns
        self.model.eval()
        
        # Generate synthetic event sequence
        events = []
        current_time = 0
        
        for _ in range(100):
            # Sample next event
            dt = np.random.exponential(0.1)
            event_type = np.random.randint(0, self.model.n_attack_types)
            current_time += dt
            events.append((current_time, event_type))
            
        # Compute log-likelihood
        with torch.no_grad():
            ll = self.model.point_process.log_likelihood(events, current_time)
            
        print(f"Point Process Log-Likelihood: {ll.item():.4f}")
        
        self.results['pp_likelihood'] = ll.item()
        
        return self.results
    
    def evaluate_real_time_adaptation(self, stream_loader, window_size=100):
        """Evaluate real-time adaptation capability"""
        print("\n=== Real-Time Adaptation ===")
        
        adapter = RealTimeAdapter(self.model, self.device)
        
        accuracies = []
        adaptation_times = []
        
        for i, (x, y) in enumerate(stream_loader):
            if i >= 1000:  # Limit evaluation
                break
                
            x, y = x.to(self.device), y.to(self.device)
            
            # Predict before adaptation
            start_time = time.time()
            mean_pred, uncertainty = adapter.predict_with_uncertainty(x[0])
            pred = torch.argmax(mean_pred)
            
            # Check accuracy
            correct = (pred == y[0]).item()
            accuracies.append(correct)
            
            # Update adapter
            adapter.update(x[0], y[0])
            
            adaptation_time = time.time() - start_time
            adaptation_times.append(adaptation_time)
            
            # Print progress
            if (i + 1) % 100 == 0:
                recent_acc = np.mean(accuracies[-window_size:])
                avg_time = np.mean(adaptation_times[-window_size:])
                print(f"Step {i+1}: Accuracy={recent_acc:.4f}, Time={avg_time*1000:.2f}ms")
                
        final_accuracy = np.mean(accuracies)
        avg_adaptation_time = np.mean(adaptation_times)
        
        print(f"\nFinal Streaming Accuracy: {final_accuracy:.4f}")
        print(f"Average Adaptation Time: {avg_adaptation_time*1000:.2f}ms")
        
        self.results['streaming_accuracy'] = final_accuracy
        self.results['adaptation_time_ms'] = avg_adaptation_time * 1000
        
        return self.results, accuracies
    
    def plot_results(self, history, streaming_acc):
        """Plot comprehensive results"""
        fig, axes = plt.subplots(2, 3, figsize=(15, 8))
        
        # Training loss
        if 'train_loss' in history:
            axes[0, 0].plot(history['train_loss'])
            axes[0, 0].set_xlabel('Epoch')
            axes[0, 0].set_ylabel('Loss')
            axes[0, 0].set_title('Training Loss')
            axes[0, 0].grid(True)
        
        # Validation accuracy
        if 'val_acc' in history:
            axes[0, 1].plot(history['val_acc'])
            axes[0, 1].set_xlabel('Epoch')
            axes[0, 1].set_ylabel('Accuracy')
            axes[0, 1].set_title('Validation Accuracy')
            axes[0, 1].grid(True)
        
        # ELBO
        if 'elbo' in history:
            axes[0, 2].plot(history['elbo'])
            axes[0, 2].set_xlabel('Iteration')
            axes[0, 2].set_ylabel('ELBO')
            axes[0, 2].set_title('Evidence Lower Bound')
            axes[0, 2].grid(True)
        
        # Streaming accuracy
        if len(streaming_acc) > 0:
            window = 50
            smoothed = pd.Series(streaming_acc).rolling(window).mean()
            axes[1, 0].plot(smoothed)
            axes[1, 0].set_xlabel('Sample')
            axes[1, 0].set_ylabel('Accuracy')
            axes[1, 0].set_title(f'Streaming Accuracy (window={window})')
            axes[1, 0].grid(True)
        
        # Performance metrics bar chart
        metrics = ['Accuracy', 'F1', 'ECE']
        values = [
            self.results.get('accuracy', 0),
            self.results.get('f1', 0),
            1 - self.results.get('ece', 1)  # Convert to calibration score
        ]
        axes[1, 1].bar(metrics, values)
        axes[1, 1].set_ylabel('Score')
        axes[1, 1].set_title('Performance Metrics')
        axes[1, 1].set_ylim([0, 1])
        
        # Temporal modeling
        t = np.linspace(0, 10, 100)
        intensity = np.exp(-t) * 0.5  # Example decay
        axes[1, 2].plot(t, intensity)
        axes[1, 2].set_xlabel('Time')
        axes[1, 2].set_ylabel('Intensity')
        axes[1, 2].set_title('Attack Intensity Function')
        axes[1, 2].grid(True)
        
        plt.tight_layout()
        plt.savefig('paper4_results.png', dpi=150)
        plt.show()

# ========================= Training Framework =========================

def train_neural_ode_pp(model, train_loader, val_loader, device, epochs=30):
    """Train Neural ODE-Point Process model"""
    
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    history = defaultdict(list)
    best_val_acc = 0
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            
            optimizer.zero_grad()
            
            # Time span for ODE
            t_span = torch.linspace(0, 1, 10).to(device)
            
            # Generate synthetic events (for demonstration)
            events = [(np.random.random(), np.random.randint(0, model.n_attack_types)) 
                     for _ in range(5)]
            
            # Compute ELBO loss
            loss = model.compute_elbo(x, y, t_span, events)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            
        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                
                t_span = torch.linspace(0, 1, 10).to(device)
                output, _, _ = model(x, t_span)
                
                preds = torch.argmax(output, dim=1)
                val_correct += (preds == y).sum().item()
                val_total += len(y)
                
        val_acc = val_correct / val_total
        
        # Update scheduler
        scheduler.step()
        
        # Save history
        history['train_loss'].append(train_loss / len(train_loader))
        history['val_acc'].append(val_acc)
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model_paper4.pt')
            
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"  Train Loss: {train_loss/len(train_loader):.4f}")
            print(f"  Val Acc: {val_acc:.4f}")
            
    return history

# ========================= Data Preparation =========================

class TimeSeriesDataset(Dataset):
    """Custom dataset for time series security data"""
    
    def __init__(self, X, y, sequence_length=10):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
        self.sequence_length = sequence_length
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ========================= Main Execution =========================

def main():
    """Main execution"""
    print("="*80)
    print("Paper 4: Neural ODE-Point Process Integration")
    print("="*80)
    
    # Load data
    print("\n1. Loading ICS3D datasets...")
    from Paper3_implementation import ICS3DDataLoader  # Reuse from Paper 3
    
    data_loader = ICS3DDataLoader()
    
    # Load dataset
    print("   Loading Edge-IIoT dataset...")
    X, y = data_loader.load_edge_iiot('DNN')
    
    # Preprocess
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    le = LabelEncoder()
    y = le.fit_transform(y)
    
    # Split data
    from sklearn.model_selection import train_test_split
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
    
    # Create datasets
    train_dataset = TimeSeriesDataset(X_train[:10000], y_train[:10000])
    val_dataset = TimeSeriesDataset(X_val[:2000], y_val[:2000])
    test_dataset = TimeSeriesDataset(X_test[:2000], y_test[:2000])
    
    # Create loaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    stream_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
    
    print(f"\n2. Data Statistics:")
    print(f"   Train samples: {len(train_dataset)}")
    print(f"   Val samples: {len(val_dataset)}")
    print(f"   Test samples: {len(test_dataset)}")
    print(f"   Feature dimension: {X.shape[1]}")
    print(f"   Number of classes: {len(np.unique(y))}")
    
    # Initialize model
    print("\n3. Initializing Neural ODE-Point Process Model...")
    model = NeuralODEPointProcess(
        input_dim=X.shape[1],
        hidden_dim=128,
        n_attack_types=len(np.unique(y))
    )
    
    print(f"   Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # Train model
    print("\n4. Training with Bayesian Neural ODE...")
    history = train_neural_ode_pp(
        model, train_loader, val_loader, device, epochs=30
    )
    
    # Variational Inference
    print("\n5. Variational Inference...")
    vi = VariationalInference(model, device)
    # Note: Skipping full VI training for computational efficiency
    # elbo_history = vi.train_svi(train_loader, epochs=10)
    
    # Comprehensive Evaluation
    print("\n6. Comprehensive Evaluation...")
    evaluator = RealTimeEvaluator(model, device)
    
    # Detection performance
    results = evaluator.evaluate_detection_performance(test_loader)
    
    # Uncertainty calibration
    results = evaluator.evaluate_uncertainty_calibration(test_loader)
    
    # Temporal performance
    results = evaluator.evaluate_temporal_performance(test_loader)
    
    # Real-time adaptation
    results, streaming_acc = evaluator.evaluate_real_time_adaptation(stream_loader)
    
    # Plot results
    print("\n7. Generating visualizations...")
    evaluator.plot_results(history, streaming_acc)
    
    # Final summary
    print("\n" + "="*80)
    print("FINAL RESULTS SUMMARY")
    print("="*80)
    print(f"Detection Accuracy: {results['accuracy']:.4f}")
    print(f"F1 Score: {results['f1']:.4f}")
    print(f"Calibration Error (ECE): {results['ece']:.4f}")
    print(f"Streaming Accuracy: {results['streaming_accuracy']:.4f}")
    print(f"Adaptation Time: {results['adaptation_time_ms']:.2f}ms")
    print(f"Point Process Log-Likelihood: {results['pp_likelihood']:.4f}")
    print("="*80)
    
    return model, history, results

if __name__ == "__main__":
    model, history, results = main()

